### Twitter API: 
    https://www.safaribooksonline.com/library/view/mining-the-social/9781449368180/ch01.html
### Source:
    alexlitel.github.io/congresstweets/
    https://archive.org/details/archiveteam-twitter-stream-2017-11
    http://www.abs.gov.au/websitedbs/a3121120.nsf/home/statistical+language+-+describing+frequencies

In [1]:
#import csv, json
import os, json, csv
from collections import Counter
import pandas as pd


counter = 0
cwd = os.getcwd()
data_subfolder = "tweets"
full_path = cwd +"/"+ data_subfolder


## Before Pandas

In [2]:
# #iterate through all the files in the folder(1)
# for root_folder, subfolders, files in os.walk(full_path):
#     for file in files:
#         if ".json" in file: 
#             print(file)

In [3]:
# def parse_json_tweet(tweet):
#     #intiailsing a list to return to variable
#     tal = []
    
#     #append dictionary item to list:
#     tal.append(tweet['id'])
#     tal.append(tweet['screen_name'])
#     tal.append(tweet['user_id'])
#     tal.append(tweet['time'])
#     tal.append(tweet['text'])
#     tal.append(tweet['source'])
    
#     return tal

In [4]:
# #target file:
# target_file = open('output.csv', 'w')
# #create a csv writer object:
# csv_file = csv.writer(target_file)

# #open file:
# for root_folder, subfolders, files in os.walk(full_path):
#     for file in files:
#         try:
#             f = open(root_folder+'/'+ file, 'r',encoding='utf-8')
#             json_data = json.load(f)
#             for tweet in json_data[0:10]:
#                 tweet_to_list = parse_json_tweet(tweet)
#                 csv_file.writerow(tweet_to_list)

#     #replacing with a fuction:
# #                 for key, val in tweet.items():
# #                     data_row.append(val)
# #                     csv_file.writerow(data_row)

#         except Exception as e:
#             print("Please Check your file path. Error: {}".format(e))
            
# target_file.close()

## After Pandas

In [5]:
def parse_json_tweet(indexObj):
    tweet_to_list = []
    tweet_data = ['created_at', 'text']
    user_data = ['name', 'screen_name', 'followers_count',
                'friends_count', 'favourites_count', 
                'statuses_count','created_at']

    for tweet_content in tweet_data:
        tweet_to_list.append(indexObj[tweet_content])
    if indexObj['place'] != None:
        tweet_to_list.append(indexObj['place']['full_name'])
    else:
        tweet_to_list.append(None)
    for user in user_data:
        tweet_to_list.append(indexObj['user'][user])
    
    return tweet_to_list

In [6]:
#return list of tuples(location, abs frequency, rel frequency)
def most_common(dfObj,category_type,length):
    counts = Counter(df[category_type])
    total = df[category_type].shape[0]
    most_common = []
    #modifying the Counter.most_common() function to include relative Frequency
    for l, c in counts.most_common(11):
        rel = c/total
        most_common.append((l,c,rel))
        
    return most_common

In [7]:
def text_length(dfObj):
    adding_up = 0
    count_text = Counter(df.text.str.len())
    total = df['text'].shape[0]
    length_list = []
    for lenTxt, count in count_text.most_common():
        percent = (count/total) * 100
        adding_up += percent
        length_list.append((lenTxt,count, percent))
    print('total of all percentages: {:.2f}%'.format(adding_up))
    return length_list

In [8]:
labels = ['created_at', 'text', 'place','name', 'screen_name', 'followers_count',
        'friends_count', 'favourites_count', 'statuses_count','created_at']

#target file:
target_file = open('output.csv', 'w')
#create a csv writer object:
csv_file = csv.writer(target_file)
#write header:
csv_file.writerow(labels)
for root_folder, subfolders, files in os.walk(full_path):
     for file in files:
        try:
            #read json file with pandas:
            df = pd.read_json(root_folder+'/'+file ,lines = True)
            #iterate through each tweet and extract relavent data:
            for i in range(df.shape[0]):
                # if the entry is a delete the size == 24;
                if(df.loc[i]['delete'].__sizeof__() > 24 ):
                    counter += 1
                    continue
                else: 
                    tweet_to_list = parse_json_tweet(df.loc[i])
                    #write to file:
                    csv_file.writerow(tweet_to_list)
        except Exception as e:
            print("Please Check your file path. Error: {}".format(e))
            
#close file:
target_file.close()
print("Removed {} deleted tweets from dataset.".format(counter))
    


Removed 10147 deleted tweets from dataset.


In [9]:
df = pd.read_csv('output.csv')
df.head()

,created_at,text,place,name,screen_name,followers_count,friends_count,favourites_count,statuses_count,created_at.1
0,2018-01-01 06:00:00,🎯.تابع من قام بالتفضيل والرتويت✅🚀 \n🎯.Follow ...,NaN,Follow Help🎯SNIPER,sniper1kh,16143,15212,25313,14124,Mon May 22 13:10:23 +0000 2017
1,2018-01-01 06:00:00,RT @alwaysbaejy: อิพี่คนนี้ทำไมไม่มีความอ่อนโย...,NaN,마이배🐺,formy101boys,11,249,790,16476,Sun Oct 30 11:50:36 +0000 2016
2,2018-01-01 06:00:00,#HappyNewYear everybody!!!,NaN,Deven Parikh,DParikh30,51,432,7,23422,Wed Jan 02 19:56:34 +0000 2013
3,2018-01-01 06:00:00,@Ignaciorojod la vida es dura carnaaa,NaN,.,alexasxx,593,391,3854,24556,Mon Aug 15 19:46:06 +0000 2011
4,2018-01-01 06:00:00,おうって…,NaN,長谷川ふみ,hasegawahumi,860,645,6095,288809,Thu Sep 12 09:52:42 +0000 2013


In [10]:
df.shape

(15262, 10)

In [11]:
#Summary statistics for numerical columns
df.describe()

,followers_count,friends_count,favourites_count,statuses_count
count,1.526200e+04,15262.000000,1.526200e+04,1.526200e+04
mean,6.163255e+03,1447.087603,1.119269e+04,4.212280e+04
std,9.712170e+04,10305.720756,3.207172e+04,1.708952e+05
min,0.000000e+00,0.000000,0.000000e+00,1.000000e+00
25%,6.900000e+01,110.000000,2.360000e+02,2.290000e+03
50%,2.660000e+02,286.000000,2.303500e+03,1.064600e+04
75%,8.217500e+02,694.000000,1.005850e+04,3.516425e+04
max,4.771269e+06,628048.000000,1.210022e+06,8.293085e+06


In [12]:
#correlations between columns in a dataframe
df.corr()

,followers_count,friends_count,favourites_count,statuses_count
followers_count,1.000000,0.209095,-0.003294,0.392220
friends_count,0.209095,1.000000,0.035797,0.098911
favourites_count,-0.003294,0.035797,1.000000,0.173139
statuses_count,0.392220,0.098911,0.173139,1.000000


In [13]:
#all tweets with geo location:
df[df.place.notnull()].head()

,created_at,text,place,name,screen_name,followers_count,friends_count,favourites_count,statuses_count,created_at.1
19,2018-01-01 06:00:00,oficialmente morta depois desse ultimo ep de b...,"Goiânia, Brasil",barbra,barbaradavif,423,392,6493,12675,Wed Nov 30 21:55:23 +0000 2011
58,2018-01-01 06:00:01,"Wind 1.0 mph W. Barometer 30.489 in, Steady. T...","Wapakoneta, OH",Auglaize County EMA,TroyAnderson6,465,302,66,22661,Mon Aug 15 11:41:58 +0000 2011
67,2018-01-01 06:00:01,La solución a todo es un caramelo halls o ment...,"Central, Paraguay",Deysi Chilavert,_chilavertdeysi,1972,2002,1189,27244,Fri Mar 29 18:48:23 +0000 2013
76,2018-01-01 06:00:01,@dera_aldosari المشكلة ان جماعتك وعيال عمك كله...,"الرياض, المملكة العربية السعودية",الزعيم الملكي,CaesarHilal,204,1069,1193,7141,Mon Jun 10 11:22:20 +0000 2013
126,2018-01-01 06:00:02,Bienvenido 2018 😎\n#cabala \n#laurelennlacart...,"Viña del Mar, Chile",☆..Florcita Ibarcena,Ibarce2Florcita,84,136,90,545,Thu Nov 12 07:19:13 +0000 2015


#### Based on the available geo location tweets, I am showing the frequecy data(top 10).

In [14]:
pd.DataFrame(most_common(df,'place',11), columns=['Location','Absolute Frequency', 'Relative Frequency']).dropna()

,Location,Absolute Frequency,Relative Frequency
1,"Rio de Janeiro, Brasil",7,0.000459
2,"Houston, TX",5,0.000328
3,"Chicago, IL",5,0.000328
4,"Los Angeles, CA",4,0.000262
5,"Atlanta, GA",3,0.000197
6,"Dallas, TX",3,0.000197
7,"الرياض, المملكة العربية السعودية",2,0.000131
8,"Wisconsin, USA",2,0.000131
9,"Guayaquil, Ecuador",2,0.000131
10,"Nashville, TN",2,0.000131


#### Parsing through all the text item for the length and find the absolute frequency and percents

In [15]:
pd.DataFrame(text_length(df),columns=['Length of text', 'Absolute Frequency','Percentage']).head(10)

total of all percentages: 100.00%


,Length of text,Absolute Frequency,Percentage
0,140,2320,15.201153
1,139,439,2.876425
2,23,216,1.415280
3,39,160,1.048355
4,35,145,0.950072
5,41,140,0.917311
6,28,140,0.917311
7,38,139,0.910759
8,52,138,0.904207
9,30,137,0.897654


### Didn't get to:
Doing the sentiment anaylsis of the text to see if the key words are found in the text:
key_words['excited','2018']


example: "excited about 2018!". how many times has this been favourites_count?

word frequency; --> youtube.

##### required external library for sentiment anaylsis, nltk api (http://text-processing.com/demo/sentiment/)
